In [11]:
import os
import torch
import numpy as np
import torch.nn as nn
import ikrlib as ikrl
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import Augmentor

In [14]:
from image import CustomDataset, SmallCNNMultiClass, train

if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu" 

train_x = np.empty((0,80,80,3))
train_y = np.empty((0),dtype=int)

test_x = np.empty((0,80,80,3))
test_y = np.empty((0),dtype=int)

for i in range(1,CLASSES+1):
    train_i = np.array(list(ikrl.png_load(os.path.join("train",str(i)), False).values()))
    label_i = np.full(len(train_i),i-1)
    train_x = np.concatenate((train_x, train_i), axis=0)
    train_y = np.concatenate((train_y, label_i), axis=0)

    train_i = np.array(list(ikrl.png_load(os.path.join("dev",str(i)), False).values()))
    label_i = np.full(len(train_i),i-1)
    train_x = np.concatenate((train_x, train_i), axis=0)
    train_y = np.concatenate((train_y, label_i), axis=0)

    test_i = np.array(list(ikrl.png_load(os.path.join("dev",str(i)), False).values()))
    label_i = np.full(len(test_i),i-1)
    test_x = np.concatenate((test_x, test_i), axis=0)
    test_y = np.concatenate((test_y, label_i), axis=0)

print("Images were successfully loaded")

# convert 80,80,3 to 3,80,80
train_x = np.array(train_x)
train_x = np.transpose(train_x, (0, 3, 1, 2))

test_x = np.array(test_x)
test_x = np.transpose(test_x, (0, 3, 1, 2))

# Convert NumPy arrays to PyTorch tensors
train_tensors = torch.Tensor(train_x)
test_tensors = torch.Tensor(test_x)


# Create new TensorDataset instances with the modified labels
train_dataset = CustomDataset(train_tensors, train_y)
test_dataset = CustomDataset(test_tensors, test_y)
print("Dataset was successfully created")

model = SmallCNNMultiClass()
criterion = F.cross_entropy
model = model.to(dev)
# model = Net().to(dev)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
accuracys, losses = train(model, train_dataset, test_dataset, optimizer, criterion, dev, 500)

Images were successfully loaded
Dataset was successfully created
Epoch: 1/500, Loss: 54.9538, Accuracy: 0.0806, 5 and 62
Epoch: 51/500, Loss: 46.1196, Accuracy: 0.5968, 37 and 62
Epoch: 101/500, Loss: 44.8162, Accuracy: 0.6452, 40 and 62
Epoch: 151/500, Loss: 43.6482, Accuracy: 0.7258, 45 and 62
Epoch: 201/500, Loss: 43.6658, Accuracy: 0.8065, 50 and 62
Epoch: 251/500, Loss: 43.1446, Accuracy: 0.8226, 51 and 62
Epoch: 301/500, Loss: 43.9178, Accuracy: 0.8387, 52 and 62
Epoch: 351/500, Loss: 43.5401, Accuracy: 0.9032, 56 and 62
Epoch: 401/500, Loss: 43.1020, Accuracy: 0.9355, 58 and 62
Epoch: 451/500, Loss: 43.2152, Accuracy: 0.9355, 58 and 62
Epoch: 500/500, Loss: 43.3870, Accuracy: 0.9355, 58 and 62


([0.08064516129032258,
  0.5967741935483871,
  0.6451612903225806,
  0.7258064516129032,
  0.8064516129032258,
  0.8225806451612904,
  0.8387096774193549,
  0.9032258064516129,
  0.9354838709677419,
  0.9354838709677419],
 [54.95383930206299,
  46.11960744857788,
  44.81620764732361,
  43.648189544677734,
  43.665766954422,
  43.14463543891907,
  43.91777539253235,
  43.54011631011963,
  43.10203433036804,
  43.2152361869812])

In [8]:
from torch.utils.data import TensorDataset

test_x = np.empty((0,80,80,3))
test_x = np.array(list(ikrl.png_load('eval', False).values()))
test_x = np.transpose(test_x, (0, 3, 1, 2))

test_dataset = TensorDataset(torch.Tensor(test_x))
test_loader = DataLoader(test_dataset, batch_size=736)

for x in test_loader:
    pred = model(x[0].to(dev))
    _, pred = torch.max(pred, dim=1)
pred = pred + 1
print(pred)

tensor([23, 27, 23,  1, 20, 22,  2,  2, 23, 13, 17, 21, 24, 30, 10, 18, 15, 26,
        30,  3, 30,  2,  1, 27, 14,  1,  2, 28, 24, 27,  9, 12,  5, 20, 14, 11,
         6, 14, 20, 12, 25, 28, 19, 27, 11, 22, 12,  4,  9,  7, 23, 29, 28, 23,
        28,  2, 25,  6, 15, 17,  9, 10,  2, 19, 24, 26,  2, 31, 23,  2, 28, 27,
         2, 24, 10,  4, 22, 26,  9, 31, 20,  1, 13, 11, 20, 10, 28, 16,  9, 20,
        23, 18,  6, 17, 18, 13, 15,  4, 27,  6,  5,  6, 12, 27,  1, 15, 14,  5,
        10, 17, 27, 24, 11,  6, 25, 27, 10,  2, 11,  3, 10,  9,  5, 16, 26, 25,
         2, 16, 28, 12, 27,  4, 22, 28,  2,  6,  1, 14,  6, 21,  2,  1,  2, 22,
        27, 29,  1, 25,  4, 31,  3,  6, 20, 16, 21, 13,  1, 10, 27,  5,  5, 19,
        21,  7,  6, 23, 12, 24,  6, 12, 15, 20, 14, 13, 18, 12,  9,  4, 15, 30,
        21, 20, 12, 11, 20, 23, 26, 16,  4, 27,  4, 17, 17,  6,  1, 11, 25, 17,
        17,  6, 14, 30, 21, 27, 24, 31, 10, 16, 15, 10, 24, 20,  2, 12,  5, 28,
        26,  2, 23, 18, 15,  1, 13,  7, 

## SVM

In [ ]:
from svm import SVMTrain
from sklearn.model_selection import train_test_split

def png_load(dir_name):
    """
    Loads all *.png images from directory dir_name into a dictionary. Keys are the file names
    and values and 2D numpy arrays with corresponding grayscale images
    """
    features = {}
    for f in glob(dir_name + '/*.png'):
        features[f] = np.array(Image.open(f), dtype=np.float64)
    return features


train_x = np.empty((0,80,80,3))
train_y = np.empty((0),dtype=int)

for i in range(1,CLASSES+1):
    train_i = np.array(list(ikrl.png_load(os.path.join("train",str(i)), True).values()))
    label_i = np.full(len(train_i),i-1)
    train_x = np.concatenate((train_x, train_i), axis=0)
    train_y = np.concatenate((train_y, label_i), axis=0)

print("Images were successfully loaded")

train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.3,random_state=109)

train_dataset = CustomDataset(train_x, train_y)

model = SVMTrain()
model.train(train_dataset, CustomDataset(test_x, test_y))
print(model.predict(test_x[0]))

## GMM

## Audio

In [10]:
import ikrlib as ilib

from audio import audio_adjust, reduce_noise, data_augumentation, pre_emphasis, min_max_normalize, compute_deltas, cepstral_mean_subtraction

cepstral_mean_subtraction_enabled = False
delta_coefficients_enabled = False
coefficients_normalization = False

audio_adjust_enabled = True
reduce_noise_enabled = True
data_augmentation_enabled = True
data_pre_emphasis = False

if audio_adjust_enabled:
    for i in range(1, 32):
        audio_adjust(ilib.get_directory(f"train/{i}"))
        audio_adjust(ilib.get_directory(f"dev/{i}"))
    
print("Silence was successfully removed")


if reduce_noise_enabled:
    for i in range(1, 32):
        reduce_noise(ilib.get_directory(f"train/{i}", audio_adjust_enabled))
        reduce_noise(ilib.get_directory(f"dev/{i}", audio_adjust_enabled))
    print("Noise was successfully removed")

if data_augmentation_enabled:
    for i in range(1, 32):
        data_augumentation(ilib.get_directory(f"train/{i}", audio_adjust_enabled, reduce_noise_enabled))
    print("Data augumentation was done")

if data_pre_emphasis:
    train = {}
    dev = {}
    for i in range(1, 32):
        train[i] =  np.vstack(pre_emphasis(ilib.get_directory(f'train/{i}', audio_adjust_enabled, reduce_noise_enabled, data_augmentation_enabled)))
        dev[i] =  list(pre_emphasis(ilib.get_directory(f'dev/{i}', audio_adjust_enabled, reduce_noise_enabled)))
    print("Pre emphasis was successfull")

if not data_pre_emphasis:
    train = {}
    dev = {}
    for i in range(1, 32):
        train[i] = np.vstack(list(ilib.wav16khz2mfcc(ilib.get_directory(f'train/{i}', audio_adjust_enabled, reduce_noise_enabled, data_augmentation_enabled)).values()))
        dev[i] = list(ilib.wav16khz2mfcc(ilib.get_directory(f'train/{i}', audio_adjust_enabled, reduce_noise_enabled)).values())
    print("Loading data was successful")

if coefficients_normalization:
    for i in range(1, 32):
        train[i] = min_max_normalize(train[i])

if delta_coefficients_enabled:
    for i in range(1, 32):
        train_delta_coeffs = compute_deltas(train[i], window_size=2)
        train_derivative_delta_coeffs = compute_deltas(train[i], window_size=2)
        train[i] = np.concatenate((train[i], train_delta_coeffs, train_derivative_delta_coeffs), axis=1)

if cepstral_mean_subtraction_enabled:
    for i in range(1, 32):
        train[i] = cepstral_mean_subtraction(train[i])

M = 3  # Počet gaussovských komponent
MUs = {}
COVs = {}
Ws = {}
for i in range(1, 32):
    MUs[i] = train[i][np.random.randint(1, len(train[i]), M)]  # Počiatočna stredná hodnota
    #COVs[i] = [np.cov(train[i].T)] * M  # Počiatočna kovariančná matica
    COVs[i] = [np.diag(np.diag(np.cov(train[i].T))) for _ in range(M)]  # Initial diagonal covariance matrix
    Ws[i] = np.ones(M) / M

for jj in range(30):
    # TTL_t je doveryhodnosť
    for i in range(1, 32):
        Ws[i], MUs[i], COVs[i], TTL = ilib.train_gmm(train[i], Ws[i], MUs[i], COVs[i])
        print(f'Iteration: {jj} Total log likelihood: {TTL} for person {i}')

P_t=0.5
P_n=1.0-P_t

score = []
correct = 0
total = 0

for true_class in range(1, 32):
    for dev_p_i in dev[true_class]:
        dev_p_i_cpy = dev_p_i.copy()

        if coefficients_normalization:
            dev_p_i_cpy = min_max_normalize(dev_p_i_cpy)

        if delta_coefficients_enabled:
            test_t_delta_coeffs = compute_deltas(dev_p_i_cpy, window_size=2)
            test_t_derivative_delta_coeffs = compute_deltas(test_t_delta_coeffs, window_size=2)

            dev_p_i_cpy = np.concatenate((dev_p_i_cpy, test_t_delta_coeffs, test_t_derivative_delta_coeffs), axis=1)

        if cepstral_mean_subtraction_enabled:
            dev_p_i_cpy = cepstral_mean_subtraction(dev_p_i_cpy)

        # Compute the likelihoods for all the classes
        likelihoods = np.array([ilib.logpdf_gmm(dev_p_i_cpy, Ws[i], MUs[i], COVs[i]).sum() for i in range(1, 32)])

        # Find the class with the highest likelihood
        predicted_class = np.argmax(likelihoods) + 1

        # Compare the predicted class with the true class
        if predicted_class == true_class:
            correct += 1
        total += 1

accuracy = correct / total
print(f"Fraction of correctly recognized targets: {accuracy * 100}%")

Removing silence from records in directory train/1/
Removing silence from records in directory dev/1/
Removing silence from records in directory train/2/
Removing silence from records in directory dev/2/
Removing silence from records in directory train/3/
Removing silence from records in directory dev/3/
Removing silence from records in directory train/4/
Removing silence from records in directory dev/4/
Removing silence from records in directory train/5/
Removing silence from records in directory dev/5/
Removing silence from records in directory train/6/
Removing silence from records in directory dev/6/
Removing silence from records in directory train/7/
Removing silence from records in directory dev/7/
Removing silence from records in directory train/8/
Removing silence from records in directory dev/8/
Removing silence from records in directory train/9/
Removing silence from records in directory dev/9/
Removing silence from records in directory train/10/
Removing silence from records